# Acesso a bases de dados com Python  
Pedro Cardoso

(ISE/UAlg - pcardoso@ualg.pt)

## Exemplos de conexões a bases de dados usando um Connector/Python

### MySQL 

o método  `connect()` constructor creates a connection to the MySQL server and returns a `MySQLConnection` object.
(see https://dev.mysql.com/doc/connector-python/en/connector-python-connectargs.html for other argument options)

Notas 1: verificar que todos módulos necessários estão instalados, nomeadamente, pode de ter de instalar o `mysql-connector`

In [ ]:
!pip install mysql-connector 

Nota 2: verificar que têm um servidor de MySQL a correr no `host` que indicarem, com utilizador e password corretas, i.e., para os exemplos abaixo, existe uma base de dados chamada `sensors` a qual o utilizador `sensors` com password `##sensors##` tem acesso.

![php_my_admin_new_user.png](php_my_admin_new_user.png)

In [ ]:
import mysql.connector

cnx = mysql.connector.connect(user='sensors', 
                              password='##sensors##',
                              host='localhost', # replace 'localhost', if necessary
                              database='sensors')
cnx

Algumas informações sobre a conexão podem ser consultadas no `__dict__`

In [ ]:
cnx.__dict__

E no final devemos libertar sempre a conexão

In [ ]:
cnx.close()

De um modo geral é aconselhável
* fazer tratamento de exceções 
* e ter um ficheiro de congiguração (config.py) 
```
config = {
    'host' : 'localhost',
    'user' : 'sensors',
    'password' : '##sensors##',
    'db' : 'sensors'
}
```
e depois...

In [ ]:
# Comecamos por importar o ficheiro de configuração

# Se correr a partir de um "script normal",  poderá depender do sistema e faze-se
# from config import config 
# from .config import config 

# em Jupyter fazemos
%run config.py

import mysql.connector

try:
    cnx = mysql.connector.connect(**config)
except mysql.connector.Error as err:
    print('Ups! Ocorreu um erro!')
    print(err)
else:
    print('Sucesso!')
    cnx.close()

**Exercício:** experimentem a desligar o servidor or mudar o user no ficheiro de configuração, etc. e vejam os erros que dá. Tratem as exceções de modo adequatado.

### SQLite
No caso do SQLite basta importar o pacote `sqlite3`
[https://docs.python.org/2/library/sqlite3.html]

Notas 1: verificar que todos módulos necessários estão instalados, nomeadamente, pode de ter de instalar o `mysql-connector`

In [ ]:
!pip install pysqlite3

In [ ]:
import sqlite3

e criar a conexão usando o método `connect`, que tem como parâmetro o caminho para o ficheiro que contém a base de dados 

In [ ]:
conn = sqlite3.connect('example.db')


## Criação de uma base de dados

Para a criação das tabelas e relacionamentos podemos construiro o sql ou, como alternativa, podemos usar ferramentas como sejam o MySQL Workbench, o Phpmyadmin, o SQlite Browser, o DataGrip, etc. 


Consideremos o caso em que contruímos o sql...

### MySQL

Comecemos por criar uma base de dados no servidor de MySQL (façam sempre tratamento de exceções...).

In [ ]:
# from config import config as conf
%run config.py
import mysql.connector

sql = '''
    CREATE SCHEMA IF NOT EXISTS `sensors` DEFAULT CHARACTER SET utf8 ;
    
    USE `sensors` ;
    
    -- -----------------------------------------------------
    -- Table `sensors`.`Location`
    -- -----------------------------------------------------
    CREATE TABLE IF NOT EXISTS `sensors`.`Location` (
      `idLocation` INT NOT NULL AUTO_INCREMENT,
      `name` VARCHAR(45) NOT NULL,
      `description` VARCHAR(45) NOT NULL,
      PRIMARY KEY (`idLocation`),
      UNIQUE INDEX `name_UNIQUE` (`name` ASC)) ENGINE = InnoDB;


    -- -----------------------------------------------------
    -- Table `sensors`.`Unit`
    -- -----------------------------------------------------
    CREATE TABLE IF NOT EXISTS `sensors`.`Unit` (
      `unit` VARCHAR(45) NOT NULL,
      `description` VARCHAR(45) NOT NULL,
      PRIMARY KEY (`unit`)) ENGINE = InnoDB;


    -- -----------------------------------------------------
    -- Table `sensors`.`Sensor`
    -- -----------------------------------------------------
    CREATE TABLE IF NOT EXISTS `sensors`.`Sensor` (
      `idSensor` INT NOT NULL AUTO_INCREMENT,
      `idLocation` INT NOT NULL,
      `name` VARCHAR(45) NOT NULL,
      `unit` VARCHAR(45) NOT NULL,
      PRIMARY KEY (`idSensor`),
      INDEX `fk_Sensor_Location_idx` (`idLocation` ASC),
      INDEX `fk_Sensor_Units1_idx` (`unit` ASC),
      UNIQUE INDEX `uniq_loc_vs_sensor` (`idLocation` ASC, `name` ASC),
      CONSTRAINT `fk_Sensor_Location`
        FOREIGN KEY (`idLocation`)
        REFERENCES `sensors`.`Location` (`idLocation`)
        ON DELETE CASCADE
        ON UPDATE CASCADE,
      CONSTRAINT `fk_Sensor_Units1`
        FOREIGN KEY (`unit`)
        REFERENCES `sensors`.`Unit` (`unit`)
        ON DELETE CASCADE
        ON UPDATE CASCADE) ENGINE = InnoDB;


    -- -----------------------------------------------------
    -- Table `sensors`.`Reading`
    -- -----------------------------------------------------
    CREATE TABLE IF NOT EXISTS `sensors`.`Reading` (
      `idReading` INT NOT NULL AUTO_INCREMENT,
      `idSensor` INT NOT NULL,
      `timestamp` TIMESTAMP NOT NULL,
      `value` FLOAT NOT NULL,
      PRIMARY KEY (`idReading`),
      INDEX `fk_Reading_Sensor1_idx` (`idSensor` ASC),
      CONSTRAINT `fk_Reading_Sensor1`
        FOREIGN KEY (`idSensor`)
        REFERENCES `sensors`.`Sensor` (`idSensor`)
        ON DELETE CASCADE
        ON UPDATE CASCADE) ENGINE = InnoDB;


    -- -----------------------------------------------------
    -- Table `sensors`.`Alert`
    -- -----------------------------------------------------
    CREATE TABLE IF NOT EXISTS `sensors`.`Alert` (
      `idAlert` INT NOT NULL AUTO_INCREMENT,
      `idSensor` INT NOT NULL,
      `timestamp` TIMESTAMP NOT NULL,
      `description` VARCHAR(45) NOT NULL,
      `cleared` BIT NULL,
      PRIMARY KEY (`idAlert`),
      INDEX `fk_Alert_Sensor1_idx` (`idSensor` ASC),
      CONSTRAINT `fk_Alert_Sensor1`
        FOREIGN KEY (`idSensor`)
        REFERENCES `sensors`.`Sensor` (`idSensor`)
        ON DELETE CASCADE
        ON UPDATE CASCADE) ENGINE = InnoDB;
'''

In [ ]:
try:
    cnx = mysql.connector.connect(**config)
    cursor = cnx.cursor()
    cursor.execute(sql)
except mysql.connector.Error as err:
    print(err)
else:
    cnx.close()

### SQLite



In [ ]:
import sqlite3

sql = '''
create table Location
(
    idLocation integer
        constraint Location_pk
            primary key autoincrement,
    name TEXT not null,
    description text not null
);


create table Unit
(
    unit text
        constraint Unit_pk
            primary key,
    description text not null
);

create table Sensor
(
    idSensor integer
        constraint Sensor_pk
            primary key,
    idLocation integer not null
        constraint Sensor_Location_idLocation_fk
            references Location
                on update cascade on delete cascade,
    name text not null,
    unit text not null
        constraint Sensor_Unit_unit_fk
            references Unit
                on update cascade on delete cascade
);

create table Reading
(
    idReading integer
        constraint Reading_pk
            primary key,
    idSensor integer
        constraint Reading_Sensor_idSensor_fk
            references Sensor
                on update cascade on delete cascade,
    timestamp datetime default CURRENT_TIMESTAMP,
    value real not null
);


create table Alert
(
    idAlert integer
        constraint Alert_pk
            primary key,
    idSensor integer
        constraint Alert_Sensor_idSensor_fk
            references Sensor
                on update cascade on delete cascade,
    description text not null,
    cleared integer

)
'''

In [ ]:
conn = sqlite3.connect('sensors.db')

cursor = conn.cursor()

# Create tables -- executescript permite correr vários comandos de uma vez 
cursor.executescript(sql)

## Operações CRUD

### `INSERT`  em MySQL 

Aberta a conexão em MySQL

In [ ]:
cnx = mysql.connector.connect(**config)
cursor = cnx.cursor()

inserir uma nova localização na base de dados e obter o id correspondente

In [ ]:
# prepare the sql query for the new location
sql = '''
INSERT INTO location 
    (idLocation, name, description) 
VALUES 
    (DEFAULT, %s, %s)
'''

data = ('Prometheus Server', 'Prometheus Server @ lab. 163 / ISE /UAlg')

#execute the sql query and get the new location id
cursor.execute(sql, data)
location_id = cursor.lastrowid
location_id

Quando estamos a usar um sistema transacional, como o InnoDB, temos de efetuar o "commit" depois de fazer um INSERT, DELETE, ou UPDATE.

In [ ]:
cnx.commit()

Inserir uma nova `Unit`

In [ ]:
# Insert a new unit (if it does not exist)
sql = '''
REPLACE INTO Unit 
    (unit, description) 
VALUES 
    ("percent", "percentage of usage")
'''
cursor.execute(sql)
cnx.commit()

Inserir um novo sensor e obter o seu id

In [ ]:
# prepare the sql query for the new sensor
sql = '''INSERT INTO `sensor` (`idSensor`, `idLocation`, `name`, `unit`)
        VALUES (DEFAULT, %(idLocation)s, %(name)s, %(unit)s);'''
data = {
        'idLocation': location_id, 
        'name' : 'cpu_sensor_01', 
        'unit' : 'percent'
       }

#execute the sql query
cursor.execute(sql, data)
sensor_id = cursor.lastrowid
cnx.commit()

E agora, obter alguns dados e enviar para a base de dados

In [ ]:
import psutil

sql = '''
INSERT INTO `reading` 
    (`idReading`, `idSensor`, `timestamp`, `value`)     
VALUES 
    (DEFAULT, %(idSensor)s, DEFAULT, %(value)s)
'''

for _ in range(20):
    data = {
           'idSensor' : sensor_id, 
            'value' : psutil.cpu_percent(interval=1)
           }
    cursor.execute(sql, data) 
    cnx.commit()
    print('.', end='')


In [ ]:
cursor.close()
cnx.close()

### `INSERT`  em SQLite

Aberta a conexão em sqlite

In [ ]:
import sqlite3

cnx = sqlite3.connect('sensors.db')
cursor = cnx.cursor()

inserir uma nova localização na base de dados e obter o id correspondente

In [ ]:
# prepare the sql query for the new location
sql = '''
INSERT INTO location 
    (name, description) 
VALUES 
    (?, ?)
'''

data = ('Prometheus Server', 'Prometheus Server @ lab. 163 / ISE /UAlg')

#execute the sql query and get the new location id
cursor.execute(sql, data)
location_id = cursor.lastrowid
location_id

Quando estamos a usar um sistema transacional, temos de efetuar o `commit` depois de fazer um `INSERT`, `DELETE`, ou `UPDATE`.

Note-se que podemos desfazer as alterações desde o último `commit` chamando o método `rollback()`

In [ ]:
cnx.commit()

Inserir uma nova `Unit`

In [ ]:
# Insert a new unit (if it does not exist)
sql = '''
REPLACE INTO Unit 
    (unit, description) 
VALUES 
    ("percent", "percentage of usage")
'''
cursor.execute(sql)
cnx.commit()

Inserir um novo sensor e obter o seu id

In [ ]:
# prepare the sql query for the new sensor
sql = '''INSERT INTO `sensor` (`idLocation`, `name`, `unit`)
        VALUES (:idLocation, :name, :unit);'''

# prepare data for the named style:
data = {
        'idLocation': location_id, 
        'name' : 'cpu_sensor_01', 
        'unit' : 'percent'
       }

#execute the sql query
cursor.execute(sql, data)
sensor_id = cursor.lastrowid
cnx.commit()

E agora, obter alguns dados e enviar para a base de dados

In [ ]:
import psutil

sql = '''
INSERT INTO `reading` 
    (`idSensor`, `value`)     
VALUES 
    (:idSensor, :value)
'''

for _ in range(20):
    data = {
           'idSensor' : sensor_id, 
            'value' : psutil.cpu_percent(interval=1)
           }
    cursor.execute(sql, data) 
    cnx.commit()
    print('.', end='')


In [ ]:
cursor.close()
cnx.close()

## Selecionar dados

### `SELECT` em MySQL 

In [ ]:
cnx = mysql.connector.connect(**config)
cursor = cnx.cursor()  

In [ ]:
sql = '''
SELECT idLocation, name, description 
FROM location 
WHERE description LIKE "%163%"'''

cursor.execute(sql)

for (idLocation, name, description) in cursor:
  print("id: {}\n\t name: {} \n\t description: {}".format(idLocation, name, description))


In [ ]:
sql = '''
SELECT idReading, idSensor, timestamp, value 
FROM reading 
WHERE value BETWEEN %s and %s
'''
data = (5, 50)

cursor.execute(sql, data)

for (idReading, idSensor, timestamp, value) in cursor:
  print("idReading: {}\n\t idSensor: {} \n\t time: {} \n\t value: {}".format(idReading, idSensor, timestamp, value))

In [ ]:
cursor.close()
cnx.close()

### `SELECT` em SQLite

In [ ]:
import sqlite3

cnx = sqlite3.connect('sensors.db')
cursor = cnx.cursor() 

In [ ]:
sql = '''
SELECT idLocation, name, description 
FROM location 
WHERE description LIKE "%163%"'''

cursor.execute(sql)

for (idLocation, name, description) in cursor:
  print("id: {}\n\t name: {} \n\t description: {}".format(idLocation, name, description))


In [ ]:
sql = '''
SELECT idReading, idSensor, timestamp, value 
FROM reading 
WHERE value BETWEEN ? and ?
'''
data = (5, 50)

cursor.execute(sql, data)

for (idReading, idSensor, timestamp, value) in cursor:
  print("idReading: {}\n\t idSensor: {} \n\t time: {} \n\t value: {}".format(idReading, idSensor, timestamp, value))

In [ ]:
sql = '''
    select *
    from Location
        inner join Sensor S on Location.idLocation = S.idLocation
        inner join Unit U on S.unit = U.unit
        inner join Reading R on S.idSensor = R.idSensor
    where value between :low and :high
    order by value
'''

data = {
    'low': 5,
    'high': 20
}

cursor.execute(sql, data)

Podemos obter os nomes das colunas

In [ ]:
cursor.description

In [ ]:
lista_de_colunas = [linha[0] for linha in cursor.description]
lista_de_colunas

In [ ]:
for linha in cursor:
    print('\t'.join([f'|{coluna}: {valor}' for valor, coluna  in zip(linha, lista_de_colunas)]))
    

Usando o comando `fetchall` podemos obter todos os resultados de uma única vez como uma lista de tuplos

In [ ]:
# é necessario voltar a correr o select pois o cursor foi esvaziado
cursor.execute(sql, data)

cursor.fetchall()

Podemos também converter para um dicionário mas __nosso caso não é boa ideia__ pois duas colunas "têm o mesmo nome" (e.g., `nome`), pelo que se perdem colunas.

In [ ]:
# é necessario voltar a correr o select pois o cursor foi esvaziado
cursor.execute(sql, data)

for linha in cursor:
    print({coluna: valor for valor, coluna  in zip(linha, lista_de_colunas)})

In [ ]:
cursor.close()
cnx.close()